In [ ]:
# initial analysis of linked trips from BATS 2023
# background:
# this analysis uses the linked trip analysis ran on 20250728
# which uses the tue to thu weighted data from February 2025 (WeightedDataset_02212025)
# the config for the trip linking process can be found in: https://github.com/ZephyrTransport/travel-diary-survey-tools/blob/df723a6376c0859b34e0fec89206c681b485353d/config/pipeline_config_mtc.toml

import shutil

# Make a copy of the linked trip file to a new analysis folder
TripLinking02b_csv = r"M:\Data\HomeInterview\Bay Area Travel Study 2023\Data\Processed\TripLinking_20250728\02b-link_trips_week\trip.csv"
TripLinking02b_csv_renamed = r"M:\Data\HomeInterview\Bay Area Travel Study 2023\Data\Processed\TripLinking_20250728\Linked_Trip_Analysis\trip_linked.csv"

shutil.copy(TripLinking02b_csv, TripLinking02b_csv_renamed)

print(f"Linked trip file copied from:\n{TripLinking02b_csv}\nto:\n{TripLinking02b_csv_renamed}")

In [ ]:
import pandas as pd
from enum import Enum

# Read the linked trip file
LinkedTrips_filepath = r"M:\Data\HomeInterview\Bay Area Travel Study 2023\Data\Processed\TripLinking_20250728\Linked_Trip_Analysis\trip_linked.csv"
LinkedTrips_df = pd.read_csv(LinkedTrips_filepath)

# Define enums for mode, path and purpose
class Mode(Enum):
    OTHER = 0
    WALK = 1
    BIKE = 2
    DA = 3
    HOV2 = 4
    HOV3 = 5
    WALKTRAN = 6
    DRIVETRAN = 7
    SCHBUS = 8
    TNC = 9

class Path(Enum):
    NONE = 0
    FULLNETWORK = 1
    NO_TOLL_NETWORK = 2 # not used in the current processing
    BUS = 3
    LRT = 4
    PREMIUM = 5
    BART = 6
    FERRY = 7

class Dpurp(Enum):
    HOME = 0
    WORK = 1
    SCHOOL = 2
    ESCORT = 3
    PERS_BUS = 4
    SHOP = 5
    MEAL = 6
    SOCREC = 7
    CHANGE_MODE = 10 # change mode is still in the dataset, but it is only a very small number of linked trips (23 cases)
    OTHER = 11
    MISSING = -1


# Map numeric codes to enum names
LinkedTrips_df["mode_enum"] = LinkedTrips_df["mode"].apply(
    lambda x: Mode(x).name if x in [m.value for m in Mode] else None
)

LinkedTrips_df["path_enum"] = LinkedTrips_df["path"].apply(
    lambda x: Path(x).name if x in [p.value for p in Path] else None
)

LinkedTrips_df["dpurp_enum"] = LinkedTrips_df["dpurp"].apply(
    lambda x: Dpurp(x).name if x in [d.value for d in Dpurp] else None
)

# View the first few rows (but make sure no indivdiual records are shown before committing to GitHub)
# LinkedTrips_df.head()

In [ ]:
modeShare_df = (
    LinkedTrips_df
    .groupby(["mode_enum"])["trip_weight"]
    .sum()
    .reset_index()
)

display(modeShare_df)

# Export to CSV
output_filepath = r"M:\Data\HomeInterview\Bay Area Travel Study 2023\Data\Processed\TripLinking_20250728\Linked_Trip_Analysis\modeShare_df.csv"
modeShare_df.to_csv(output_filepath, index=False)

print(f"File saved to: {output_filepath}")

In [ ]:
modeShare_commute_df = (
    LinkedTrips_df[
        (LinkedTrips_df["dpurp_enum"] == "WORK")
    ]
    .groupby(["mode_enum"])["trip_weight"]
    .sum()
    .reset_index()
)

display(modeShare_commute_df)

# Export to CSV
output_filepath = r"M:\Data\HomeInterview\Bay Area Travel Study 2023\Data\Processed\TripLinking_20250728\Linked_Trip_Analysis\modeShare_commute_df.csv"
modeShare_commute_df.to_csv(output_filepath, index=False)

print(f"File saved to: {output_filepath}")


In [ ]:
modeXpathXdpurp_df = (
    LinkedTrips_df
    .groupby(["mode_enum", "path_enum", "dpurp_enum"])["trip_weight"]
    .sum()
    .reset_index()
)

display(modeXpathXdpurp_df)

# Export to CSV
output_filepath = r"M:\Data\HomeInterview\Bay Area Travel Study 2023\Data\Processed\TripLinking_20250728\Linked_Trip_Analysis\modeXpathXdpurp_.csv"
modeXpathXdpurp_df.to_csv(output_filepath, index=False)

print(f"File saved to: {output_filepath}")
